# all text encoders

In [1]:
import pprint
pp = pprint.PrettyPrinter(indent=4, width=200, depth=None, stream=None, compact=False, sort_dicts=False)

In [2]:
import os
import json

with open('../pdf_to_dictionary/english_syn_ant.json', encoding='utf-8') as f:
    d = json.load(f)

In [3]:
import numpy as np
from numpy.linalg import norm

def my_cosine_similarity(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

In [4]:
# make smaller test dict
d_keys_li=list(d.keys())
smaller_dict={}
for i in range(0,10):
    smaller_dict[d_keys_li[i]]=d[d_keys_li[i]]
# pp.pprint(smaller_dict)

In [5]:
# d - full test disctionary
# smaller_dict - 10 key dictionary
# DICT = smaller_dict
DICT = d

In [6]:
for baseword,v in DICT.items():
    print(baseword,v["syn"],v["ant"])
    break

aback ['backwards', 'rearwards', 'aft', 'abaft', 'astern', 'behind', 'back'] ['onwards', 'forwards', 'ahead', 'before', 'afront', 'beyond', 'afore']


In [7]:
def get_avg_res_bert(tokenizer, model):
    from scipy.spatial import distance
    import numpy as np
    sum_syn=0
    sum_ant=0
    cnt_syn=0
    cnt_ant=0

    sum=0
    cnt=0
    avg=0

    for baseword,v in DICT.items():
        encoded_input_base=tokenizer(baseword,return_tensors='pt')
        output_base = model(**encoded_input_base)
        embedded_base_avg=np.average(output_base.last_hidden_state[0].detach().numpy(), axis=0)

        for synonym in v["syn"]:     
            try:       
                encoded_input_synonym=tokenizer(synonym,return_tensors='pt')
                output_synonym = model(**encoded_input_synonym)
                embedded_synonym_avg=np.average(output_synonym.last_hidden_state[0].detach().numpy(), axis=0)
                sim_syn=my_cosine_similarity(embedded_base_avg,embedded_synonym_avg)
                sum_syn=sum_syn+sim_syn
                cnt_syn=cnt_syn+1
            except:
                continue
        
        for antonym in v["ant"]:
            try:
                encoded_input_antonym=tokenizer(antonym,return_tensors='pt')
                output_antonym = model(**encoded_input_antonym)
                embedded_antonym_avg=np.average(output_antonym.last_hidden_state[0].detach().numpy(), axis=0)
                sim_ant=my_cosine_similarity(embedded_base_avg,embedded_antonym_avg)
                sum_ant=sum_ant+sim_ant
                cnt_ant=cnt_ant+1
            except:
                continue

        os.system('cls')            
        print(cnt_syn, cnt_ant, end="\r")
           
    avg_syn = sum_syn/cnt_syn  
    avg_ant = sum_ant/cnt_ant
    print("syn", avg_syn)
    print("ant", avg_ant)

    return avg_syn, avg_ant

In [8]:
def get_avg_res_other(tokenizer, model):
    from scipy.spatial import distance
    import numpy as np
    sum_syn=0
    sum_ant=0
    cnt_syn=0
    cnt_ant=0

    sum=0
    cnt=0
    avg=0

    for baseword,v in DICT.items():
        encoded_input_base=tokenizer(baseword,return_tensors='pt')
        output_base = model(**encoded_input_base,output_hidden_states=True)
        output_base_np=output_base.hidden_states[-1][0].detach().numpy()
        embedded_base_avg=np.average(output_base_np, axis=0)

        for synonym in v["syn"]:
            try:
                encoded_input_synonym=tokenizer(synonym,return_tensors='pt')
                output_synonym = model(**encoded_input_synonym,output_hidden_states=True)
                output_synonym_np=output_synonym.hidden_states[-1][0].detach().numpy()
                embedded_synonym_avg=np.average(output_synonym_np, axis=0)
                sim_syn=my_cosine_similarity(embedded_base_avg,embedded_synonym_avg)
                sum_syn=sum_syn+sim_syn
                cnt_syn=cnt_syn+1
            except:
                continue
        
        for antonym in v["ant"]:
            try:
                encoded_input_antonym=tokenizer(antonym,return_tensors='pt')
                output_antonym = model(**encoded_input_antonym,output_hidden_states=True)
                output_antonym_np=output_antonym.hidden_states[-1][0].detach().numpy()
                embedded_antonym_avg=np.average(output_antonym_np, axis=0)
                sim_ant=my_cosine_similarity(embedded_base_avg,embedded_antonym_avg)
                sum_ant=sum_ant+sim_ant
                cnt_ant=cnt_ant+1
            except:
                continue
        
        os.system('cls')
        print(cnt_syn, cnt_ant, end="\r")

    avg_syn = sum_syn/cnt_syn  
    avg_ant = sum_ant/cnt_ant
    print("syn", avg_syn)
    print("ant", avg_ant)

    return avg_syn, avg_ant

In [9]:
with open("results.json", "r") as resultf:
    dic_result = json.load(resultf)
dic_result

{'bert-base-multilingual-uncased': {'syn_hu': 0.722074735583503,
  'ant_hu': 0.717287334589222,
  'syn_en': 0.79346014996581,
  'ant_en': 0.7750938003989282},
 'bert-base-multilingual-cased': {'syn_hu': 0.5632927332691204,
  'ant_hu': 0.5609021603555736,
  'syn_en': 0.5289188860975226,
  'ant_en': 0.5018978259905483},
 'NYTK/text-generation-news-gpt2-small-hungarian': {'syn_hu': 0.5717219781714749,
  'ant_hu': 0.5415410494257994,
  'syn_en': 0.6182320676318803,
  'ant_en': 0.6159549340590915},
 'NYTK/text-generation-poem-petofi-gpt2-small-hungarian': {'syn_hu': 0.6306462456930798,
  'ant_hu': 0.6006082052976565,
  'syn_en': 0.6326308510037462,
  'ant_en': 0.6294172660219062},
 'SZTAKI-HLT/hubert-base-cc': {'syn_hu': 0.9410662500765817,
  'ant_hu': 0.939179996788529,
  'syn_en': 0.9354769090465983,
  'ant_en': 0.9323585145156933},
 'sberbank-ai/mGPT': {'syn_hu': 0.8669438558143153,
  'ant_hu': 0.85937414462522,
  'syn_en': 0.8728994038749214,
  'ant_en': 0.8703220840820617},
 'NYTK/sent

In [14]:
# bert-base-multilingual-uncased
# Run time: 21m
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
model = BertModel.from_pretrained("bert-base-multilingual-uncased")

avg_syn, avg_ant = get_avg_res_bert(tokenizer,model)
dic_result["bert-base-multilingual-uncased"]["syn_en"] = avg_syn
dic_result["bert-base-multilingual-uncased"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

c:\Users\L0021\Anaconda3\envs\test\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be

syn 0.79346014996581 1405 1646 1663 1667 1936 1993 2447 2482 2777 2812 2841 2946 2967 3137 3197 3510 3580 3696 3831 42164391 4496 4757 4834 5036 5236 5340 5433 5523 5556 5747 5948 5986 6016 6162 6209 6215 6218 6253 6456 6462 6590 6639 6698 6744 6925 6992 7808 8004 8042 8106 8744 9041 9044 9070 9074 9188 9232 9292 9322 9479 9621 9944 1007210073 10178 10202 1023010519 10788 10960 11095 11266 11533 11916 12091 12279 12309 12399 12635 12675 12688 12795 13306 13413 13702 13795 13804 13922 14007 14045 14107 14110 14132 14235 14284 14313 14345 14379 14402 14462 14474 14565 14874 14914 14934 15029 15236 15351 15374 15407 15564 15783 15960 15966 16089 16130 16242 16253 16265 16279 16464 16661 16663 16826 17064 17067 17144 17311 17360 17760 17816 18112 18380 18419 18422 18774 18810 18868 18894 18898 18974 19446 19567 19623 19698 19928 20315 20580 2068020873 21110 21328 21365 21624 21689 21904 22222 22276 22354 22398 22617 22731 22915 23057 23171 23179 2367523724 23988 24081
ant 0.775093800398928

In [15]:
# bert-base-multilingual-cased
# Run time: 19m
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertModel.from_pretrained("bert-base-multilingual-cased")

avg_syn, avg_ant = get_avg_res_bert(tokenizer,model)
dic_result["bert-base-multilingual-cased"]["syn_en"] = avg_syn
dic_result["bert-base-multilingual-cased"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


syn 0.528918886097522611 13741422 1634 1636 1656 1853 1929 1936 1974 2330 2447 2791 2807 2841 2967 2983 3197 3366 3369 3519 3762 3855 3862 4004 40584834 5036 5320 5340 5523 5719 57915939 6047 6055 6105 6207 6306 63336456 6459 6744 70937095 7243 7279 72807440 8004 8012 8297 8453 8493 8609 8757 9070 9134 9218 9266 9292 9337 9340 9385 958410072 10115 10122 10202 1043110470 10542 10753 10818 11059 11081 11266 11404 11526 11533 11581 11712 11872 12214 12567 12644 12675 12688 13029 13039 13091 13257 13641 13702 13760 14007 14235 14262 1426914284 14345 14462 14474 14502 14565 14616 14874 14912 14914 14934 1499315153 15236 15302 15346 15407 15664 15752 15849 15851 15960 16089 16149 16182 16217 16279 16627 16661 16819 1700717034 17289 17336 17786 17925 17996 18079 18085 18112 18380 18450 18894 19521 19561 19623 19928 19946 20112 20315 20362 20688 21110 21184 21328 21329 21359 21365 21624 21689 22171 22276 22402 22575 22631 22915 23011 23031 23037 23101 2311123158 23179 23185 23259 2326423336 23

In [16]:
# hubert-base-cc
# Run time: 18m
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("SZTAKI-HLT/hubert-base-cc")
tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")

avg_syn, avg_ant = get_avg_res_bert(tokenizer,model)
dic_result["SZTAKI-HLT/hubert-base-cc"]["syn_en"] = avg_syn
dic_result["SZTAKI-HLT/hubert-base-cc"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

Some weights of the model checkpoint at SZTAKI-HLT/hubert-base-cc were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


syn 0.93547690904659839366 804 879 880 1350 1630 1634 1646 1813 2664 28122894 2967 31973366 3510 3621 3743 3855 3862 4004 4072 4216 4496 4767 4920 4956 50365392 5537 5542 5556 5638 5748 5921 6118 6315 6333 659066316638 6817 6821 69476975 6988 7095 7231 7343 7440 7452 7820 7822 8042 8071 8241 8245 86088744 8850 8899 8903 8924 9074 9314 9322 9392 9460 9522 9564 9600 10020 10178 10376 10431 10542 11060 11077 11404 11453 11533 11572 1169611872 11881 12125 1227912309 123311242512429 12520 12609 12983 13026 13244 13330 13395 13673 13849 13879 14107 14407 14474 14553 14782 14866 14908 14910 1496614971 15014 15043 15210 15236 1523815302 15346 15374 15629 1566415752 15983 16208 16218 16246 164641682616900 16976 16998 17007 17057 17336 17360 17391 17760 17816 17854 17888 1790317921 18112 18238 18375 18380 18382 18411 18431 18450 18499 18598 18762 18868 18914 19001 19239 1936219388 19521 19561 19567 19623 19656 19698 19719 19930 20781 20873 20906 21285 21319 21328 21359 21365 21689 21743 21829 22

In [15]:
# text-generation-news-gpt2-small-hungarian
# Run time: 21m
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("NYTK/text-generation-news-gpt2-small-hungarian")
model = AutoModelForCausalLM.from_pretrained("NYTK/text-generation-news-gpt2-small-hungarian")

avg_syn, avg_ant = get_avg_res_other(tokenizer,model)
dic_result["NYTK/text-generation-news-gpt2-small-hungarian"]["syn_en"] = avg_syn
dic_result["NYTK/text-generation-news-gpt2-small-hungarian"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

syn 0.61823206763188033639 3820 4527 5420 5538 6594 7377 8003 8034 8916 9137 10508 10708 1191612664 12665 13448 1376014276 14757 14774 15210 15360 16822 17006 17136 20130 21129 22560 23079 23333 23688 24087
ant 0.6159549340590915


In [16]:
# text-generation-poem-petofi-gpt2-small-hungarian
# Run time: 21m
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("NYTK/text-generation-poem-petofi-gpt2-small-hungarian")
model = AutoModelForCausalLM.from_pretrained("NYTK/text-generation-poem-petofi-gpt2-small-hungarian")

avg_syn, avg_ant = get_avg_res_other(tokenizer,model)
dic_result["NYTK/text-generation-poem-petofi-gpt2-small-hungarian"]["syn_en"] = avg_syn
dic_result["NYTK/text-generation-poem-petofi-gpt2-small-hungarian"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

syn 0.6326308510037462 3335 5493 5720 5923 6387 6923 7082 9094 9525 9786 10348 11187 11792 11888 11982 12225 12342 13542 14379 14597 15155 15736 16199 16279 16494 16574 16584 16654 17350 18249 19866 20289 20430 20986 21246 21875 22454 22522 22547 23158 23265 23279 23696 23988
ant 0.6294172660219062


In [17]:
# mGPT
# Run time: 234m
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/mGPT")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/mGPT")

avg_syn, avg_ant = get_avg_res_other(tokenizer,model)
dic_result["sberbank-ai/mGPT"]["syn_en"] = avg_syn
dic_result["sberbank-ai/mGPT"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

syn 0.8728994038749214
ant 0.8703220840820617


In [18]:
# sentiment-hts2-xlm-roberta-hungarian
# Run time: 11m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts2-xlm-roberta-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts2-xlm-roberta-hungarian")

avg_syn, avg_ant = get_avg_res_other(tokenizer,model)
dic_result["NYTK/sentiment-hts2-xlm-roberta-hungarian"]["syn_en"] = avg_syn
dic_result["NYTK/sentiment-hts2-xlm-roberta-hungarian"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

syn 0.3351082592695616452087 2200 2484 2518 2544 2790 4157 4221 4271 4518 4624 4989 54845633 5655 6068 6269 6804 7042 7326 7340 74577676 7958 8035 8147 8727 8908 8956 8993 9076 9891 10101 10303 10380 10436 10625 11182 11543 11880 11888 12153 12162 12185 12292 12309 12407 12421 12427 12582 13075 13203 13218 13394 13738 14415 14910 15088 15370 15885 16107 16976 17068 17128 17130 17174 17433 17537 17690 17792 17809 17848 1802018502 19104 19458 19605 19929 20431 20584 20863 21131 21285 21440 21645 22451 22503 22528 23280 23339 23459 23684 24120
ant 0.3267472495863554


In [19]:
# sentiment-hts5-xlm-roberta-hungarian
# Run time: 18m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts5-xlm-roberta-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts5-xlm-roberta-hungarian")

avg_syn, avg_ant = get_avg_res_other(tokenizer,model)
dic_result["NYTK/sentiment-hts5-xlm-roberta-hungarian"]["syn_en"] = avg_syn
dic_result["NYTK/sentiment-hts5-xlm-roberta-hungarian"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

syn 0.8034598824887169 1405 1648 2480 2863 4248 5236 5720 5925 5937 6155 6356 6820 6845 7113 7307 7500 8516 8744 8968 9603 9678 9858 10017 10471 11105 11246 12155 12225 12342 12344 12399 12880 1441314729 14773 14993 16062 16231 16265 17012 17301 17339 17383 17903 18151 18589 18671 18712 18788 18887 19185 20564 20885 21046 22396 22417 22523 22561 22621 23080 23334 23694
ant 0.7818645143761956


In [10]:
# sentiment-hts5-hubert-hungarian
# Run time: 18m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts5-hubert-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts5-hubert-hungarian")

avg_syn, avg_ant = get_avg_res_other(tokenizer,model)
dic_result["NYTK/sentiment-hts5-hubert-hungarian"]["syn_en"] = avg_syn
dic_result["NYTK/sentiment-hts5-hubert-hungarian"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

c:\Users\L0021\Anaconda3\envs\test\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


syn 0.867754533185284481 1624 4309 4419 46934736 5163 5489 5556 5923 6917 7715 9071 9292 9821 9822 10045 10584 10615 11150 11389 11593 11880 11985 12117 12176 1222012256 12349 12553 13075 13237 13740 14098 14576 15239 15868 15984 16345 16360 16364 16391 16395 16583 16917 16952 17072 19000 19162 19366 20988 20992 22225 2242022632 22634 22916 22963 2364923650
ant 0.8625745139627063


In [11]:
# sentiment-hts2-hubert-hungarian
# Run time: 17m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts2-hubert-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts2-hubert-hungarian")

avg_syn, avg_ant = get_avg_res_other(tokenizer,model)
dic_result["NYTK/sentiment-hts2-hubert-hungarian"]["syn_en"] = avg_syn
dic_result["NYTK/sentiment-hts2-hubert-hungarian"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

syn 0.814264884341784820 1616 1910 2152 2565 2811 3093 3268 3323 3582 4060 4213 4271 4577 4696 5167 5349 5498 5522 5553 5964 6600 6851 6866 6987 7037 7213 7437 7571 7614 7801 7853 7961 8247 8387 8782 9297 10216 10297 10627 10791 11359 11602 11832 11921 11944 11985 12103 12379 12407 12480 12560 12564 12962 13308 13740 13791 13952 14475 14608 14892 14895 14950 15098 15178 15356 15485 16012 16186 16345 16662 16978 16981 17066 17223 18217 18332 19002 19162 19383 19403 19904 20492 20872 20988 21469 21718 22749 23340 23649 23802
ant 0.8117369347724791


In [12]:
# xlm-roberta-base
# Run time: 43m
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

avg_syn, avg_ant = get_avg_res_other(tokenizer,model)
dic_result["xlm-roberta-base"]["syn_en"] = avg_syn
dic_result["xlm-roberta-base"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

syn 0.9946075502898118
ant 0.9943620873074537


In [13]:
# xlm-roberta-large
# Run time: 82m
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")

avg_syn, avg_ant = get_avg_res_other(tokenizer,model)
dic_result["xlm-roberta-large"]["syn_en"] = avg_syn
dic_result["xlm-roberta-large"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

syn 0.9954005020458192
ant 0.9951756792555159


In [14]:
# xlm-mlm-100-1280
# Run time: 109m
from transformers import XLMTokenizer, XLMWithLMHeadModel
import torch

tokenizer = XLMTokenizer.from_pretrained("xlm-mlm-100-1280")
model = XLMWithLMHeadModel.from_pretrained("xlm-mlm-100-1280")

avg_syn, avg_ant = get_avg_res_other(tokenizer,model)
dic_result["xlm-mlm-100-1280"]["syn_en"] = avg_syn
dic_result["xlm-mlm-100-1280"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

Some weights of XLMWithLMHeadModel were not initialized from the model checkpoint at xlm-mlm-100-1280 and are newly initialized: ['transformer.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


syn 0.5294648111323731
ant 0.5129910235072613


In [15]:
# xlm-roberta-xl
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("facebook/xlm-roberta-xl")
model = AutoModelForMaskedLM.from_pretrained("facebook/xlm-roberta-xl")

avg_syn, avg_ant = get_avg_res_other(tokenizer,model)
dic_result["facebook/xlm-roberta-xl"]["syn_en"] = avg_syn
dic_result["facebook/xlm-roberta-xl"]["ant_en"] = avg_ant

with open("results.json", "w+") as resultf: json.dump(dic_result, resultf)

syn 0.9475264041841726
ant 0.9445290021461257
